# Fine-tune model with DPO

## Goal

Let's see if using DPO can create a better model.

## Imports

In [1]:
import numpy as np
import pandas as pd

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
)

from trl import DPOConfig, DPOTrainer
from datasets import Dataset

## Load model

In [2]:
model_path = '/home/gbarbadillo/data/deepseekmath'
config = AutoConfig.from_pretrained(model_path)
config.gradient_checkpointing = True

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map='auto',
    torch_dtype="auto", #torch.bfloat16 does not show speed differences
    trust_remote_code=True,
    quantization_config=None,
    config=config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# TODO: check pad token on prompt recovery notebook
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Load data

In [4]:
df = pd.read_csv('/mnt/hdd0/Kaggle/aimo/external_data/dpo/v0.csv')
df.head()

,prompt,chosen,rejected,problem_idx,max_prompt_length,chosen_length,rejected_length
0,You are an expert mathematical programmer. Sol...,```python\nfrom itertools import combinations\...,"```python\nfrom sympy import binomial, symbols...",0,144,123,212
1,User: John computes the sum of the elements of...,```python\nfrom sympy import *\nfrom itertools...,"```python\nfrom sympy import binomial, symbols...",0,96,188,620
2,Below is a math problem you are to solve (non ...,```python\nfrom sympy import *\nfrom itertools...,```python\nfrom sympy import binomial\n\ndef s...,0,169,230,198
3,\nUser: John computes the sum of the elements ...,```python\nfrom itertools import combinations\...,"```python\nfrom sympy import binomial, Rationa...",0,153,177,256
4,User: John computes the sum of the elements of...,```python\nfrom itertools import combinations\...,```python\nfrom sympy import *\n\ndef sum_of_s...,0,89,173,488


In [5]:
unique_problem_ids = df['problem_idx'].unique()
len(unique_problem_ids)

509

In [6]:
train_problem_ids = np.random.choice(unique_problem_ids, int(0.95 * len(unique_problem_ids)), replace=False)
train_df = df[df['problem_idx'].isin(train_problem_ids)]
test_df = df[~df['problem_idx'].isin(train_problem_ids)]
assert len(train_df) + len(test_df) == len(df)
assert set(train_df['problem_idx'].unique()).intersection(set(test_df['problem_idx'].unique())) == set()
print(len(train_df), len(test_df))

10154 507


In [7]:
train_dataset = Dataset.from_pandas(train_df[['prompt', 'chosen', 'rejected']])
eval_dataset = Dataset.from_pandas(test_df[['prompt', 'chosen', 'rejected']])

## Fine-tuning

In [8]:
from peft import LoraConfig

# LoRA config based on https://github.com/ironbar/prompt_recovery/blob/main/notebooks/014_fine-tune_mistral_v2.ipynb
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05, # 0.1, althought Vaca suggested to use 0.05 for big models
        # r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters
        r=16,
        bias="none",
        # target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [9]:
# https://www.philschmid.de/dpo-align-llms-in-2024-with-trl
# https://huggingface.co/docs/trl/main/en/dpo_trainer
args = DPOConfig(
    output_dir="/mnt/hdd0/Kaggle/aimo/experiments/18_dpo/01_dpo-first-steps",               # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=2,         # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    per_device_eval_batch_size=4,           # batch size for evaluation
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=2e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=10,
    save_steps=100,                         # when to save checkpoint
    # save_total_limit=2,                     # limit the total amount of checkpoints
    eval_strategy="steps",                  # evaluate every n steps
    eval_steps=100,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    model_init_kwargs=None,
    max_length=1024,
    max_prompt_length=300,
    beta=0.1,                               # same as huggingface documentation, default value
    loss_type="sigmoid",                    # default value in huggingface documentation
)

trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:411: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/10154 [00:00<?, ? examples/s]

Map:   0%|          | 0/507 [00:00<?, ? examples/s]

In [10]:
trainer.train()

  0%|          | 0/634 [00:00<?, ?it/s]

/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.6944, 'grad_norm': 3.203125, 'learning_rate': 7.8125e-06, 'rewards/chosen': -0.0004793582484126091, 'rewards/rejected': 4.7817306040087715e-05, 'rewards/accuracies': 0.4424999952316284, 'rewards/margins': -0.0005271749687381089, 'logps/rejected': -91.810302734375, 'logps/chosen': -74.58899688720703, 'logits/rejected': 12.51860237121582, 'logits/chosen': 10.896106719970703, 'epoch': 0.04}
{'loss': 0.688, 'grad_norm': 4.15625, 'learning_rate': 1.5625e-05, 'rewards/chosen': -0.022095371037721634, 'rewards/rejected': -0.0355038158595562, 'rewards/accuracies': 0.5475000143051147, 'rewards/margins': 0.013408441096544266, 'logps/rejected': -92.0755844116211, 'logps/chosen': -77.1809310913086, 'logits/rejected': 12.603387832641602, 'logits/chosen': 11.228765487670898, 'epoch': 0.08}
{'loss': 0.6682, 'grad_norm': 3.84375, 'learning_rate': 1.9981627325622104e-05, 'rewards/chosen': -0.1730453372001648, 'rewards/rejected': -0.24043823778629303, 'rewards/accuracies': 0.5975000262260437, 

  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.5959993004798889, 'eval_runtime': 388.3928, 'eval_samples_per_second': 1.305, 'eval_steps_per_second': 0.327, 'eval_rewards/chosen': -0.11285519599914551, 'eval_rewards/rejected': -0.39046603441238403, 'eval_rewards/accuracies': 0.7467191815376282, 'eval_rewards/margins': 0.2776108384132385, 'eval_logps/rejected': -103.41634368896484, 'eval_logps/chosen': -81.8982162475586, 'eval_logits/rejected': 10.685792922973633, 'eval_logits/chosen': 9.107816696166992, 'epoch': 0.16}


...

  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.3919623792171478, 'eval_runtime': 374.6222, 'eval_samples_per_second': 1.353, 'eval_steps_per_second': 0.339, 'eval_rewards/chosen': -1.1435599327087402, 'eval_rewards/rejected': -2.9309442043304443, 'eval_rewards/accuracies': 0.8156168460845947, 'eval_rewards/margins': 1.787384033203125, 'eval_logps/rejected': -128.8211212158203, 'eval_logps/chosen': -92.20526123046875, 'eval_logits/rejected': 6.475827217102051, 'eval_logits/chosen': 4.985001087188721, 'epoch': 0.95}


/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /home/gbarbadillo/data/deepseekmath - will assume that the vocabulary was not modified.
  warnings.warn(
/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'loss': 0.2465, 'grad_norm': 2.546875, 'learning_rate': 1.230030851695263e-08, 'rewards/chosen': -0.30363237857818604, 'rewards/rejected': -2.697334051132202, 'rewards/accuracies': 0.9225000143051147, 'rewards/margins': 2.3937015533447266, 'logps/rejected': -119.35763549804688, 'logps/chosen': -77.18180847167969, 'logits/rejected': 7.547821521759033, 'logits/chosen': 5.730653285980225, 'epoch': 0.98}
{'train_runtime': 17792.9492, 'train_samples_per_second': 0.571, 'train_steps_per_second': 0.036, 'train_loss': 0.3783860379589093, 'epoch': 1.0}


TrainOutput(global_step=634, training_loss=0.3783860379589093, metrics={'train_runtime': 17792.9492, 'train_samples_per_second': 0.571, 'train_steps_per_second': 0.036, 'total_flos': 0.0, 'train_loss': 0.3783860379589093, 'epoch': 0.9990151664368722})

## TODO

- [x] Count the tokens in the train set, max length and max prompt length
- [x] Do the training parameters from the example have sense?
- [ ] Shuffle the data?
- [ ] Verify that the model generates text correctly, on the prompt recovery challenge after fine-tuning the model forgot to use the eos token.